# ADS Project 4: Machine Learning Fairness
## Spring 2022

Group 5: Chang Lu, Jiaxin Yu, Marcus Loke, Xiran Lin, Zaigham Khan

## Table of Contents
1. [Overview](#overview)
2. [Load Modules and Data](#load)
3. [Logistic Regression](#lr)
4. [SVM](#svm)
5. [Information Theoretic Measures for Fairness-Aware Feature Selection (FFS)](#ffs)
6. [Evaluation](#eval)
7. [References](#ref)

## 1. Overview <a class="anchor" id="overview"></a>

+ The cleaned COMPAS dataset is provided in `../output/compas-scores-two-years(cleaned).csv`. The EDA and cleaning process is described in `../doc/eda_cleaning.html` and `eda_cleaning.Rmd`.


+ Our team focused on three algorithms aimed at ensuring machine learning fairness. The algorithms are: maximizing accuracy under fairness constraints using C-LR and C-SVM (A2) and information theoretic measures for fairness-aware feature selection (FFS) (A7).


+ Note that `utils.py` and `loss_funcs.py` are needed for **Section 3 and 4**: Maximizing accuracy under fairness constraints (C-SVM and C-LR) while `utils2.py` is needed for **Section 5**: Information theoretic measures for fairness-aware feature selection (FFS).

## 2. Load Modules and Data <a class="anchor" id="load"></a>

In [1]:
# Load modules
import os, sys
import numpy as np
import pandas as pd
import utils as ut
import loss_funcs as lf
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.metrics import log_loss
from utils2 import *

In [2]:
# Load data
df = pd.read_csv('../output/compas-scores-two-years(cleaned).csv')
df.head()

,sex,age_cat,race,priors_count,c_charge_degree,length_of_stay,two_year_recid
0,Male,25 - 45,African-American,-0.733607,F,-0.177294,1
1,Male,< 25,African-American,0.055928,F,-0.350235,1
2,Male,25 - 45,Caucasian,2.029767,F,-0.254156,1
3,Female,25 - 45,Caucasian,-0.733607,M,-0.311803,0
4,Male,< 25,Caucasian,-0.536224,F,-0.350235,1


Encode categorical variables with dummy variables:
+ `sex`: 1 for male and 0 for female
+ `age_cat`: 2 for > 45, 1 for 25 - 45 and 0 for < 25
+ `race`: 1 for caucasian and 0 for african-american
+ `c_charge_degree`: 1 for F and 0 for M

In [3]:
# Encode variables with dummy variables
df['sex'] = df['sex'].apply(lambda sex: 0 if sex == 'Female' else 1)
df['age_cat'] = df['age_cat'].apply(lambda age_cat: 2 if age_cat == '> 45' else(1 if age_cat == '25 - 45' else 0))
df['race'] = df['race'].apply(lambda race: 0 if race == 'African-American' else 1)
df['c_charge_degree'] = df['c_charge_degree'].apply(lambda c_charge_degree: 0 if c_charge_degree == 'M' else 1)
df.head()

,sex,age_cat,race,priors_count,c_charge_degree,length_of_stay,two_year_recid
0,1,1,0,-0.733607,1,-0.177294,1
1,1,0,0,0.055928,1,-0.350235,1
2,1,1,1,2.029767,1,-0.254156,1
3,0,1,1,-0.733607,0,-0.311803,0
4,1,0,1,-0.536224,1,-0.350235,1


Create a function to process the data to obtain the target variable, the sensitive attribute and the remaining dataframe with the remaining features. We also perform a shuffle so that we can split the data into train and test sets.

In [4]:
# Vars to store features
features = ['sex', 'age_cat', 'priors_count', 'c_charge_degree', 'length_of_stay']
sensitive = 'race'
target = 'two_year_recid'

# Function to process and shuffle data
def process_df(df):
    y_label = df[target]
    protected_attr = df[sensitive]
    df_new = df[features]
    y_label, protected_attr, df_new = shuffle(y_label, protected_attr, df_new, random_state = 617)
    
    return y_label.to_numpy(), protected_attr.to_numpy(), df_new.to_numpy()

# Split data into train and test
y_label, protected_attr, df_new =  process_df(df)
train_index = int(len(df_new) * 0.7)
x_train, y_train, race_train = df_new[:train_index], y_label[:train_index], protected_attr[:train_index]
x_test, y_test, race_test = df_new[train_index:], y_label[train_index:],protected_attr[train_index:]

We also created a function to determine the p-rule (p%) and a function to compute calibration.

+ **Protected**: Caucasians (i.e., `race == 1`)
+ **Not protected**: African-Americans (i.e., `race == 0`)

In [5]:
# Function to compute p-rule
def p_rule(sensitive_var, y_pred):
    protected = np.where(sensitive_var == 1)[0]
    not_protected = np.where(sensitive_var == 0)[0]
    protected_pred = np.where(y_pred[protected] == 1)
    not_protected_pred = np.where(y_pred[not_protected] == 1)
    protected_percent = protected_pred[0].shape[0]/protected.shape[0]
    not_protected_percent = not_protected_pred[0].shape[0]/not_protected.shape[0]
    ratio = min(protected_percent/not_protected_percent, not_protected_percent/protected_percent)
    
    return ratio, protected_percent, not_protected_percent

In [6]:
# Function to compute calibration
def calibration(sensitive_var, y_pred, y_true):
    protected_point = np.where(sensitive_var == 1)[0]
    y_predcau = y_pred[protected_point]
    y_truecau = y_true[protected_point]
    pcau = sum(y_predcau==y_truecau)/len(y_truecau)
    not_protected_point = np.where(sensitive_var == 0)[0]
    y_predafa = y_pred[not_protected_point]
    y_trueafa = y_true[not_protected_point]
    pafa = sum(y_predafa==y_trueafa)/len(y_trueafa)
    calibration = abs(pcau-pafa)
    return(calibration)

## 3. Logistic Regression <a class="anchor" id="lr"></a>

### 3.1 Training unconstrained classifier

First we train a baseline, unconstained classifier to evaluate its accuracy and p-rule.

In [7]:
# Train model and print results
clf = LogisticRegression(random_state = 0).fit(x_train, y_train)
coeff = clf.coef_
intercept = clf.intercept_
optimal_loss = log_loss(y_train, clf.predict_proba(x_train))
results_lr = {"Classifier": ["LR", "LR"], 
              "Set": ["Train", "Test"],
              "Accuracy (%)": [clf.score(x_train, y_train)*100, clf.score(x_test, y_test)*100],
              "P-rule (%)": [p_rule(race_train, clf.predict(x_train))[0]*100, p_rule(race_test, clf.predict(x_test))[0]*100],
              "Protected (%)": [p_rule(race_train, clf.predict(x_train))[1]*100, p_rule(race_test, clf.predict(x_test))[1]*100],
              "Not protected (%)": [p_rule(race_train, clf.predict(x_train))[2]*100, p_rule(race_test, clf.predict(x_test))[2]*100],
              "Calibration (%)": [calibration(race_train, clf.predict(x_train), y_train)*100, calibration(race_test, clf.predict(x_test), y_test)*100]}
pd.DataFrame(results_lr)

,Classifier,Set,Accuracy (%),P-rule (%),Protected (%),Not protected (%),Calibration (%)
0,LR,Train,66.932367,53.771942,29.312425,54.512490,1.482763
1,LR,Test,64.957746,61.642720,33.888889,54.976303,1.005793


### 3.2 Optimizing classifier accuracy subject to fairness constraints

Now we optimize accuracy subject to fairness constraints. The details can be found in Section 3.2 of the paper on [Fairness Constraints: Mechanisms for Fair Classification](https://arxiv.org/abs/1507.05259). Notice that setting `{'race': 0}` means that the classifier should achieve 0 covariance between the sensitive feature (`race`) value and distance to the decision boundary. A 0 covariance would mean no correlation between the two variables.

In [8]:
# Setting flags
apply_fairness_constraints = 1 # set this flag to 1 since we want to optimize accuracy subject to fairness constraints
apply_accuracy_constraint = 0
sep_constraint = 0
gamma = None
sensitive_attrs = ['race']
sensitive_attrs_to_cov_thresh = {'race': 0}
x_control = {'race': race_train}

# Train model
np.random.seed(100)
w = ut.train_model(x_train,
                   y_train,
                   x_control,
                   lf._logistic_loss,
                   apply_fairness_constraints,
                   apply_accuracy_constraint,
                   sep_constraint,
                   sensitive_attrs,
                   sensitive_attrs_to_cov_thresh,
                   gamma)

In [9]:
# Fit coefficients/weights into logistic regression in sklearn
m = LogisticRegression()
m.coef_= w.reshape((1,-1))
m.intercept_ = 0
m.classes_ = np.array([0, 1])

In [10]:
# Print results
results_clr = {"Classifier": ["C-LR", "C-LR"],
               "Set": ["Train", "Test"],
               "Accuracy (%)": [m.score(x_train, y_train)*100, m.score(x_test, y_test)*100],
               "P-rule (%)": [p_rule(race_train, m.predict(x_train))[0]*100, p_rule(race_test, m.predict(x_test))[0]*100],
               "Protected (%)": [p_rule(race_train, m.predict(x_train))[1]*100, p_rule(race_test, m.predict(x_test))[1]*100],
               "Not protected (%)": [p_rule(race_train, m.predict(x_train))[2]*100, p_rule(race_test, m.predict(x_test))[2]*100],
               "Calibration (%)": [calibration(race_train, m.predict(x_train), y_train)*100, calibration(race_test, m.predict(x_test), y_test)*100]}
pd.DataFrame(results_clr)

,Classifier,Set,Accuracy (%),P-rule (%),Protected (%),Not protected (%),Calibration (%)
0,C-LR,Train,48.454106,99.939857,99.819059,99.879130,14.021048
1,C-LR,Test,46.084507,99.955856,99.861111,99.905213,9.535940


## 4. Support Vector Machine (SVM) <a class="anchor" id="svm"></a>

The following SVM codes are an adaptation of the paper on [Fairness Constraints: Mechanisms for Fair Classification](https://arxiv.org/abs/1507.05259). Additional helper functions like `SVM_scratch.py`, `datapreprocess.py` and `helper.py`were adapted from this GitHub [repo](https://github.com/SreeranjaniD/Fairness-in-Classification-using-SVM).

In [11]:
# Load modules
from SVM_scratch import *
from datapreprocess import *
from helper import *

# Set variables needed for training
x_control_train = {'race': race_train}
x_control_test = {'race': race_test}

Here we define a function for the classifier, which trains the model based on the fairness constraints.

In [12]:
# Define function for classifier
def classifier(apply_fairness_constraints,loss_function,c,sensitive_attrs,max_iter=1000,epoch=50,lamb=1,lr=0.1,C=1,gamma=None):
    svm =SVM()
    w = svm.training(x_train,y_train, x_control_train, loss_function,C,max_iter,lamb,epoch,lr, apply_fairness_constraints, sensitive_attrs, c,gamma)
    train_score, test_score, correct_answers_train, correct_answers_test = ut.check_accuracy(w, x_train, y_train, x_test, y_test, None, None)
    distances_hyperplane_test = (np.dot(x_test, w)).tolist()
    all_class_labels_assigned_test = np.sign(distances_hyperplane_test)
    correlation_test_dict = ut.get_correlations(None, None, all_class_labels_assigned_test, x_control_test, sensitive_attrs)
    cov_dict_test = ut.print_covariance_sensitive_attrs(None, x_test, distances_hyperplane_test, x_control_test, sensitive_attrs)
    ut.print_classifier_fairness_stats([test_score],correlation_test_dict, [cov_dict_test], sensitive_attrs[0])

### 4.1 Training unconstrained SVM classifier

In [13]:
# Train model and print results
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

svm_model = SVC(kernel = 'linear', probability = True)

# Train model and print results
clf = svm_model.fit(x_train, y_train)
optimal_loss = log_loss(y_train, clf.predict_proba(x_train))
results_svm = {"Classifier": ["SVM", "SVM"],
               "Set": ["Train", "Test"],
               "Accuracy (%)": [clf.score(x_train, y_train)*100, clf.score(x_test, y_test)*100],
               "P-rule (%)": [p_rule(race_train, clf.predict(x_train))[0]*100, p_rule(race_test, clf.predict(x_test))[0]*100],
               "Protected (%)": [p_rule(race_train, clf.predict(x_train))[1]*100, p_rule(race_test, clf.predict(x_test))[1]*100],
               "Not protected (%)": [p_rule(race_train, clf.predict(x_train))[2]*100, p_rule(race_test, clf.predict(x_test))[2]*100],
               "Calibration (%)": [calibration(race_train, clf.predict(x_train), y_train)*100, calibration(race_test, clf.predict(x_test), y_test)*100]}
pd.DataFrame(results_svm)

,Classifier,Set,Accuracy (%),P-rule (%),Protected (%),Not protected (%),Calibration (%)
0,SVM,Train,66.835749,52.006786,26.296743,50.564061,1.120395
1,SVM,Test,65.183099,60.848593,30.972222,50.900474,2.262375


### 4.2 Optimizing SVM classifier accuracy subject to fairness constraints

Now we optimize accuracy subject to fairness constraints. 

In [14]:
# Subject to fairness constraints
apply_fairness_constraints = 1 # set this flag to one since we want to optimize accuracy subject to fairness constraints
loss_function = lf._hinge_loss
sensitive_attrs = ['race']
c = {'race': 0} # covariance threshold
C = 1 # penalty term

# gamma controls how much loss in accuracy we are willing to incur to achieve fairness 
# Increase in gamma decrease the accuracy to a certain limit 
# But we are setting it to None as we are not tuning gamma
gamma = None
epochs = 1000 # Number of epochs 
lamb = 1 # lambda 
lr = 0.1  # learning rate 
max_iter = 1000

We defined a `csvm` function that returns the train and test scores, and the predicted values for train and test so that we can compute the accuracy and calibration scores later.

In [15]:
# Define function for training that returns scores
def csvm(x_train, y_train, x_test, y_test, C, max_iter, lamb, epochs,lr, apply_fairness_constraints, 
         sensitive_attrs, sensitive_attrs_to_cov_thresh=c, gamma=None):
    
    svm = SVM()
    w = svm.training(x_train, y_train, x_control_train, loss_function, C, max_iter, lamb, epochs, lr, 
                     apply_fairness_constraints, sensitive_attrs, sensitive_attrs_to_cov_thresh, gamma)
    y_test_predicted = np.sign(np.dot(x_test, w))
    y_train_predicted = np.sign(np.dot(x_train, w))
    
    def get_accuracy(y, Y_predicted):
        correct_answers = (Y_predicted == y).astype(int) # will have 1 when the prediction and the actual label match
        accuracy = float(sum(correct_answers)) / float(len(correct_answers))
        return accuracy, sum(correct_answers)

    train_score, correct_answers_train = get_accuracy(y_train, y_train_predicted)
    test_score, correct_answers_test = get_accuracy(y_test, y_test_predicted)
    return train_score, test_score, correct_answers_train, correct_answers_test, y_test_predicted, y_train_predicted

In [16]:
# Run model
train_score, test_score, correct_answers_train, correct_answers_test, y_test_predicted, y_train_predicted = csvm(x_train, y_train, x_test, y_test, C, max_iter, lamb, epochs,lr, apply_fairness_constraints, sensitive_attrs, sensitive_attrs_to_cov_thresh=c, gamma=None)

Running custom model with fairness constraints


In [17]:
# Print results
results_csvm = {"Classifier": ["C-SVM", "C-SVM"],
                "Set": ["Train", "Test"],
                "Accuracy (%)": [train_score*100, test_score*100],
                "P-rule (%)": [p_rule(race_train, y_train_predicted)[0]*100, p_rule(race_test, y_test_predicted)[0]*100],
                "Protected (%)": [p_rule(race_train, y_train_predicted)[1]*100, p_rule(race_test, y_test_predicted)[1]*100],
                "Not protected (%)": [p_rule(race_train, y_train_predicted)[2]*100, p_rule(race_test, y_test_predicted)[2]*100],
                "Calibration (%)": [calibration(race_train, y_train_predicted, y_train)*100, calibration(race_test, y_test_predicted, y_test)*100]}
pd.DataFrame(results_csvm)

,Classifier,Set,Accuracy (%),P-rule (%),Protected (%),Not protected (%),Calibration (%)
0,C-SVM,Train,48.405797,99.939857,99.819059,99.879130,14.041072
1,C-SVM,Test,46.028169,99.955856,99.861111,99.905213,9.441153


## 5. Information Theoretic Measures for Fairness-Aware Feature Selection (FFS) <a class="anchor" id="ffs"></a>

This is another method to deal with machine learning fairness and we recreated the framework that was described in the paper, [Information Theoretic Measures for Fairness-aware Feature selection (FFS)](https://arxiv.org/abs/2106.00772). In short, from the joint statistics of the data, the framework proposes that two information theoretic measures can be used to quantify the accuracy and discrmination aspect for each subset of the feature space. We then compute the Shapley coefficients for each feature to capture its effect on the sensitive/protected group (i.e., `race`).

In [18]:
# Load data 
df2 = pd.read_csv('../data/compas-scores-two-years.csv')

# Split data into target variable, sensitive variables and the other features
train_set = set_split_train(process_df2(df2)[0], process_df2(df2)[1], process_df2(df2)[2])

In [19]:
# Compute Shapley coefficients and display results sorted from highest discrimination
accuracy, discriminate = shapley_Cal(train_set)[0], shapley_Cal(train_set)[1]
shapley_results = shapley_df(discriminate,accuracy)
shapley_results

,Feature,Shapley Discrimination,Shapley Accuracy
0,priors_count,25508.281363,1.264251
1,length_of_stay,25483.034007,1.048422
2,age_cat,21627.423734,1.096104
3,sex,20962.580750,0.941318
4,c_charge_degree,20764.750822,1.036236


Based on the results, `priors_count` has the greatest impact on discrimination, but it also has the greatest effect on accuracy, which may be a problem if we removed it from the classifier. On the other hand, `length_of_stay` has relatively high discrimination but it does not have a serious impact on accuracy when compared with the rest; removing it seems like a good choice. Therefore, the feature set to be used for the LR and SVM below would be:

+ `sex`
+ `age_cat`
+ `priors_count`
+ `c_charge_degree`

In [20]:
# Vars to store features (removed 'length_of_stay')
features = ['sex', 'age_cat', 'priors_count', 'c_charge_degree']
sensitive = 'race'
target = 'two_year_recid'

# Split data into train and test
y_label, protected_attr, df_new =  process_df(df)
train_index = int(len(df_new) * 0.7)
x_train, y_train, race_train = df_new[:train_index], y_label[:train_index], protected_attr[:train_index]
x_test, y_test, race_test = df_new[train_index:], y_label[train_index:],protected_attr[train_index:]

### 5.1 Logistic regression using features from FFS

In [21]:
# Train model and print results
clf = LogisticRegression(random_state = 0).fit(x_train, y_train)
coeff = clf.coef_
intercept = clf.intercept_
optimal_loss = log_loss(y_train, clf.predict_proba(x_train))
results_ffs_lr = {"Classifier": ["FFS-LR", "FFS-LR"],
                  "Set": ["Train", "Test"],
                  "Accuracy (%)": [clf.score(x_train, y_train)*100, clf.score(x_test, y_test)*100],
                  "P-rule (%)": [p_rule(race_train, clf.predict(x_train))[0]*100, p_rule(race_test, clf.predict(x_test))[0]*100],
                  "Protected (%)": [p_rule(race_train, clf.predict(x_train))[1]*100, p_rule(race_test, clf.predict(x_test))[1]*100],
                  "Not protected (%)": [p_rule(race_train, clf.predict(x_train))[2]*100, p_rule(race_test, clf.predict(x_test))[2]*100],
                  "Calibration (%)": [calibration(race_train, clf.predict(x_train), y_train)*100, calibration(race_test, clf.predict(x_test), y_test)*100]}
pd.DataFrame(results_ffs_lr)

,Classifier,Set,Accuracy (%),P-rule (%),Protected (%),Not protected (%),Calibration (%)
0,FFS-LR,Train,67.053140,54.25751,30.036188,55.358582,1.684213
1,FFS-LR,Test,65.070423,61.94468,34.583333,55.829384,0.115192


### 5.2 SVM using features from FFS

In [22]:
svm_model = SVC(kernel = 'linear', probability = True)

# Train model and print results
clf = svm_model.fit(x_train, y_train)
optimal_loss = log_loss(y_train, clf.predict_proba(x_train))
results_ffs_svm = {"Classifier": ["FFS-SVM", "FFS-SVM"],
                   "Set": ["Train", "Test"],
                   "Accuracy (%)": [clf.score(x_train, y_train)*100, clf.score(x_test, y_test)*100],
                   "P-rule (%)": [p_rule(race_train, clf.predict(x_train))[0]*100, p_rule(race_test, clf.predict(x_test))[0]*100],
                   "Protected (%)": [p_rule(race_train, clf.predict(x_train))[1]*100, p_rule(race_test, clf.predict(x_test))[1]*100],
                   "Not protected (%)": [p_rule(race_train, clf.predict(x_train))[2]*100, p_rule(race_test, clf.predict(x_test))[2]*100],
                   "Calibration (%)": [calibration(race_train, clf.predict(x_train), y_train)*100, calibration(race_test, clf.predict(x_test), y_test)*100]}
pd.DataFrame(results_ffs_svm)

,Classifier,Set,Accuracy (%),P-rule (%),Protected (%),Not protected (%),Calibration (%)
0,FFS-SVM,Train,66.859903,52.990595,28.950543,54.63336,1.965515
1,FFS-SVM,Test,65.577465,61.602509,33.750000,54.78673,0.664165


## 6. Evaluation <a class="anchor" id="eval"></a>

In this final section, we evaluate the performance of each model using **accuracy** and **calibration**. Although there are other metrics listed below, we will only select the best model using the two mentioned metrics; the other metrics like p-rule are listed for discussion purposes. All the scores below are on the test set.

In [23]:
# Results summary
results_summary = pd.concat([pd.DataFrame(results_lr).iloc[1,], pd.DataFrame(results_clr).iloc[1,],
                             pd.DataFrame(results_svm).iloc[1,], pd.DataFrame(results_csvm).iloc[1,],
                             pd.DataFrame(results_ffs_lr).iloc[1,], pd.DataFrame(results_ffs_svm).iloc[1,]], axis=1)
results_summary.columns = results_summary.iloc[0]
results_summary = results_summary.iloc[1:,:]
results_summary

Classifier,LR,C-LR,SVM,C-SVM,FFS-LR,FFS-SVM
Set,Test,Test,Test,Test,Test,Test
Accuracy (%),64.957746,46.084507,65.183099,46.028169,65.070423,65.577465
P-rule (%),61.64272,99.955856,60.848593,99.955856,61.94468,61.602509
Protected (%),33.888889,99.861111,30.972222,99.861111,34.583333,33.75
Not protected (%),54.976303,99.905213,50.900474,99.905213,55.829384,54.78673
Calibration (%),1.005793,9.53594,2.262375,9.441153,0.115192,0.664165


We will evaluate the models using accuracy and calibration as defined by the project guidelines. Based on that, FFS-LR and FFS-SVM performed best in calibration and accuracy. Both the FFS models had similar accuracy and calibration scores, but we will select FFS-LR as the model of choice since it is more interpretable and simpler to implement as compared to SVM.

Of course, if parity (or p-rule) is the metric of choice for making model selection, then we would recommend the constrained models because they performed well on p-rule (close to 100%), albeit with a trade-off in accuracy and calibration. To note, the constrained models did not perform so well on calibration, which is theoretically possible because the constraints were optimizing parity (i.e., p-rule) and not calibration. As can be seen from the results, the p-rule for both constrained models has been optimized close to 100% because of this.

Lastly, although the FFS feature selection portion took slightly longer to run than the other models (~10 sec), it is not very detrimental to our project due to the small feature space. It might, however, have a greater impact on larger datasets with more rows and columns.

## 7. References <a class="anchor" id="ref"></a>

+ https://towardsdatascience.com/optimization-with-scipy-and-application-ideas-to-machine-learning-81d39c7938b8
+ https://github.com/mbilalzafar/fair-classification/tree/master/disparate_impact
+ https://www.propublica.org/datastore/dataset/compas-recidivism-risk-score-data-and-analysis
+ https://arxiv.org/abs/2106.00772
+ https://github.com/SreeranjaniD/Fairness-in-Classification-using-SVM